## Create Data

In [1]:
# Sample sentences for the list
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "She opened her book and started to read.",
    "Today is a sunny day.",
    "The cat sat on the mat.",
    "He loves to play soccer.",
    "I need to buy some groceries.",
    "Python programming is fun.",
    "The lake is deep and beautiful.",
    "She sings beautifully.",
    "They are planning a trip to Paris."
]

# Print the list of sentences
for sentence in sentences:
    print(sentence)

The quick brown fox jumps over the lazy dog.
She opened her book and started to read.
Today is a sunny day.
The cat sat on the mat.
He loves to play soccer.
I need to buy some groceries.
Python programming is fun.
The lake is deep and beautiful.
She sings beautifully.
They are planning a trip to Paris.


## Installation

In [2]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


## Grab Secrets

In [4]:
from google.colab import userdata

## Embeddings

In [60]:
from openai import OpenAI
from typing import List

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def list_to_nums(sentences: List[str]) -> List[List[float]]:
    """
    Converts a list of sentences into a list of numerical embeddings using OpenAI's embedding model.

    Args:
    - sentences (List[str]): A list of sentences (strings).

    Returns:
    - List[List[float]]: A list of lists of numerical embeddings.
    """

    # Initialize the list to store embeddings
    embeddings = []

    # Loop through each sentence to convert to embeddings
    for sentence in sentences:
        # Use the OpenAI API to get embeddings for the sentence

        response = client.embeddings.create(
            input=sentence,
            model="text-embedding-3-small"
        )

        embeddings.append(response.data[0].embedding)

    return embeddings

In [61]:
%%time

# Call the function
query_database = list_to_nums(sentences)

CPU times: user 75.7 ms, sys: 8.03 ms, total: 83.7 ms
Wall time: 5.57 s


In [62]:
prompt = "What is the meaning of life?"

In [63]:
%%time

prompt_embed_ = list_to_nums([prompt])
print(type(prompt_embed_), len(prompt_embed_[0]))

<class 'list'> 1536
CPU times: user 7.04 ms, sys: 0 ns, total: 7.04 ms
Wall time: 278 ms


## Quantized Influence Measure (QIM)

In [78]:
import numpy as np
from typing import Any, Dict, List, Tuple, Union


def quantize_to_kbit(arr: Union[np.ndarray, Any], k: int = 16) -> np.ndarray:
    """Converts an array to a k-bit representation by normalizing and scaling its values.

    Args:
        arr (Union[np.ndarray, Any]): The input array to be quantized.
        k (int): The number of levels to quantize to. Defaults to 16 for 4-bit quantization.
    Returns:
        np.ndarray: The quantized array with values scaled to 0 to k-1.
    """
    if not isinstance(arr, np.ndarray):  # Check if input is not a numpy array
        arr = np.array(arr)  # Convert input to a numpy array
    arr_min = arr.min()  # Calculate the minimum value in the array
    arr_max = arr.max()  # Calculate the maximum value in the array
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)  # Normalize array values to [0, 1]
    return np.round(normalized_arr * (k - 1)).astype(int)  # Scale normalized values to 0-(k-1) and convert to integer


def quantized_influence(arr1: np.ndarray, arr2: np.ndarray, k: int = 16, use_dagger: bool = False) -> Tuple[float, List[float]]:
    """
    Calculates a weighted measure of influence based on quantized version of input arrays and optionally applies a transformation.

    Args:
        arr1 (np.ndarray): First input array to be quantized and analyzed.
        arr2 (np.ndarray): Second input array to be quantized and used for influence measurement.
        k (int): The quantization level, defaults to 16 for 4-bit quantization.
        use_dagger (bool): Flag to apply a transformation based on local averages, defaults to False.
    Returns:
        Tuple[float, List[float]]: A tuple containing the quantized influence measure and an optional list of transformed values based on local estimates.
    """
    # Quantize both arrays to k levels
    arr1_quantized = quantize_to_kbit(arr1, k)
    arr2_quantized = quantize_to_kbit(arr2, k)

    # Find unique quantized values in arr1
    unique_values = np.unique(arr1_quantized)

    # Compute the global average of quantized arr2
    total_samples = len(arr2_quantized)
    y_bar_global = np.mean(arr2_quantized)

    # Compute weighted local averages and normalize
    weighted_local_averages = [(np.mean(arr2_quantized[arr1_quantized == val]) - y_bar_global)**2 * len(arr2_quantized[arr1_quantized == val])**2 for val in unique_values]
    qim = np.sum(weighted_local_averages) / (total_samples * np.std(arr2_quantized))  # Calculate the quantized influence measure

    if use_dagger:
        # If use_dagger is True, compute local estimates and map them to unique quantized values
        local_estimates = [np.mean(arr2_quantized[arr1_quantized == val]) for val in unique_values]
        daggers = {unique_values[i]: v for i, v in enumerate(local_estimates)}  # Map unique values to local estimates

        def find_val_(i: int) -> float:
            """Helper function to map quantized values to their local estimates."""
            return daggers[i]

        # Apply transformation based on local estimates
        daggered_values = list(map(find_val_, arr1_quantized))
        return qim, daggered_values
    else:
        # If use_dagger is False, return the original quantized arr1 values
        daggered_values = arr1_quantized.tolist()
        return qim

In [79]:
len(prompt_embed_[0]), len(query_database[0])

(1536, 1536)

In [80]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=16, use_dagger=False)
print(current_qim)

1.6904069806905613
CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 2.73 ms


In [81]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=3, use_dagger=False)
print(current_qim)

0.09942754196352802
CPU times: user 5.27 ms, sys: 962 µs, total: 6.23 ms
Wall time: 12.6 ms


In [82]:
len(query_database)

10

In [88]:
%%time


scores = [[sentences[i], query_database[i], quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)] for i in range(len(query_database))]
print(scores)

[['The quick brown fox jumps over the lazy dog.', [-0.01839698851108551, -0.007250950671732426, 0.003669881960377097, -0.05420133098959923, -0.022748827934265137, 0.03699697554111481, 0.029079928994178772, 0.023852646350860596, 0.011215819045901299, -0.02063000202178955, -0.013436145149171352, 0.009084305725991726, -0.022596577182412148, 0.03544909134507179, -0.016138598322868347, 0.009141399525105953, -0.011672571301460266, -0.05623134598135948, -0.030627813190221786, -0.021721133962273598, -0.018282799050211906, -0.0004706298350356519, 0.0019602307584136724, -0.0014503487618640065, -0.0006502383621409535, -0.008843242190778255, 0.006496039684861898, -0.03128756582736969, 0.008519708178937435, -0.0021854352671653032, 0.03341908007860184, -0.029435180127620697, -0.011342694982886314, -0.06470664590597153, -0.010987442918121815, -0.032530948519706726, -0.03988974541425705, -0.012668546289205551, -0.02212713658809662, -0.009008180350065231, 0.0008246925426647067, -0.0616108775138855, 0.0

In [89]:
import pandas as pd

In [94]:
%%time

refs = pd.DataFrame(scores)
refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})

CPU times: user 1.42 ms, sys: 0 ns, total: 1.42 ms
Wall time: 1.41 ms


In [96]:
refs.sort_values(by="qim", ascending=False)

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035932271275669336, -0.023653700947761536...",0.731892
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.437797
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.161556
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.122665
0,The quick brown fox jumps over the lazy dog.,"[-0.01839698851108551, -0.007250950671732426, ...",0.099428
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.052105
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.036736
8,She sings beautifully.,"[0.0009941509924829006, -0.03694251552224159, ...",0.017012
2,Today is a sunny day.,"[-0.0004934690659865737, -0.04250173643231392,...",0.014332
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.009255


## Query Search

In [107]:
import pandas as pd
from typing import List

def query_search(prompt: str) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [108]:
query_search("What is the meaning of life?")

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035932271275669336, -0.023653700947761536...",0.731892
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.437797
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.161556
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.122665
0,The quick brown fox jumps over the lazy dog.,"[-0.01839698851108551, -0.007250950671732426, ...",0.099428
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.052105
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.036736
8,She sings beautifully.,"[0.0009941509924829006, -0.03694251552224159, ...",0.017012
2,Today is a sunny day.,"[-0.0004934690659865737, -0.04250173643231392,...",0.014332
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.009255


In [109]:
query_search("The lake is dup and beauti")

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035932271275669336, -0.023653700947761536...",1.446752
8,She sings beautifully.,"[0.0009941509924829006, -0.03694251552224159, ...",0.269725
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.152648
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.112794
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.087260
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.086900
0,The quick brown fox jumps over the lazy dog.,"[-0.01839698851108551, -0.007250950671732426, ...",0.081475
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.055286
2,Today is a sunny day.,"[-0.0004934690659865737, -0.04250173643231392,...",0.054503
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.015750


In [110]:
query_search("The planning to paris is not good")

,sentences,query_embeddings,qim
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",1.996888
0,The quick brown fox jumps over the lazy dog.,"[-0.01839698851108551, -0.007250950671732426, ...",0.193268
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.153301
8,She sings beautifully.,"[0.0009941509924829006, -0.03694251552224159, ...",0.096750
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.046311
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.030507
7,The lake is deep and beautiful.,"[-0.0035932271275669336, -0.023653700947761536...",0.012530
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.009558
2,Today is a sunny day.,"[-0.0004934690659865737, -0.04250173643231392,...",0.004062
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.003036


## Read Data

In [112]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [115]:
from typing import List, Tuple
import PyPDF2

def read_and_textify(files: List[str]) -> Tuple[List[str], List[str]]:
    """
    Reads PDF files from given paths and extracts text from each page.

    This function iterates over a list of PDF file paths, opens each file, extracts text from each page,
    and compiles a list of texts and corresponding source information.

    Args:
    files (List[str]): A list of paths to PDF files.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two lists:
        1. A list of strings, where each string is the text extracted from a PDF page.
        2. A list of strings indicating the source of each text (file name and page number).
    """

    text_list = []  # List to store extracted text
    sources_list = []  # List to store source information

    # Iterate over each file path
    for file_path in files:
        # Open the PDF file
        with open(file_path, "rb") as file:
            pdfReader = PyPDF2.PdfReader(file)  # Create a PDF reader object
            # Iterate over each page in the PDF
            for i in range(len(pdfReader.pages)):
                pageObj = pdfReader.pages[i]  # Get the page object
                text = pageObj.extract_text()  # Extract text from the page
                text_list.append(text)  # Add extracted text to the list
                # Extract the file name from the path and append the source info
                sources_list.append(f"{file_path.split('/')[-1]}_page_{i}")

    return text_list, sources_list


In [117]:
%%time

uploaded_files = "/content/Understanding_and_Improving_Use_Tax_Compliance__A_Theory_of_Plann.pdf"

# Process the uploaded files to extract text and source information
textify_output = read_and_textify([uploaded_files])

# Separate the output into documents (text) and their corresponding sources
documents, sources = textify_output

CPU times: user 849 ms, sys: 4.34 ms, total: 853 ms
Wall time: 862 ms


In [126]:
type(documents), type(sources)

(list, list)

In [123]:
documents[0]

'Univ ersity of South Florida Univ ersity of South Florida \nScholar Commons Scholar Commons \nGraduate Theses and Disser tations Graduate School \n7-9-2009 \nUnderstanding and Impr oving Use-T ax Compliance: A Theor y of Understanding and Impr oving Use-T ax Compliance: A Theor y of \nPlanned Beha vior Appr oach Planned Beha vior Appr oach \nChrist opher Rober t Jones \nUniv ersity of South Florida \nFollow this and additional works at: https:/ /scholar commons.usf.edu/etd \n Part of the American Studies Commons \nScholar Commons Citation Scholar Commons Citation \nJones, Christ opher Rober t, "Understanding and Impr oving Use-T ax Compliance: A Theor y of Planned \nBeha vior Appr oach " (2009). Graduate Theses and Disser tations. \nhttps:/ /scholar commons.usf.edu/etd/2032 \nThis Disser tation is br ought t o you for fr ee and open access b y the Gr aduate School at Scholar Commons. It has \nbeen accepted for inclusion in Gr aduate Theses and Disser tations b y an authoriz ed adminis

In [125]:
len(documents), len(sources)

(113, 113)

In [127]:
documents[0], sources[0]

('Univ ersity of South Florida Univ ersity of South Florida \nScholar Commons Scholar Commons \nGraduate Theses and Disser tations Graduate School \n7-9-2009 \nUnderstanding and Impr oving Use-T ax Compliance: A Theor y of Understanding and Impr oving Use-T ax Compliance: A Theor y of \nPlanned Beha vior Appr oach Planned Beha vior Appr oach \nChrist opher Rober t Jones \nUniv ersity of South Florida \nFollow this and additional works at: https:/ /scholar commons.usf.edu/etd \n Part of the American Studies Commons \nScholar Commons Citation Scholar Commons Citation \nJones, Christ opher Rober t, "Understanding and Impr oving Use-T ax Compliance: A Theor y of Planned \nBeha vior Appr oach " (2009). Graduate Theses and Disser tations. \nhttps:/ /scholar commons.usf.edu/etd/2032 \nThis Disser tation is br ought t o you for fr ee and open access b y the Gr aduate School at Scholar Commons. It has \nbeen accepted for inclusion in Gr aduate Theses and Disser tations b y an authoriz ed admini

In [128]:
%%time

# Call the function
query_database = list_to_nums(documents)

CPU times: user 650 ms, sys: 41.8 ms, total: 692 ms
Wall time: 34.7 s


In [131]:
import pandas as pd
from typing import List

def query_search(prompt: str, sentences: list[str], query_database: list[list[float]], sources: list[str]) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            sources[i],  # Source of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "page no", 3: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [136]:
%%time

ref_tab = query_search("pful for understanding federal income", documents, query_database, sources)

CPU times: user 78.8 ms, sys: 4.82 ms, total: 83.7 ms
Wall time: 435 ms


In [137]:
ref_tab.to_json()

'{"sentences":{"15":"7 \\n variables (e.g., penalty and a udit rate) that help to expl ain and potentially improve, \\nfederal income tax compliance.  Thus, while  these reviews are he lpful for understanding \\nfederal income tax compliance,  they do not encompass all po tential taxes since different \\ntaxes could make different fact ors more or less salient.   \\nUse and Sales-Tax Compliance \\nMany states collect taxes on purchases made  by residents in the form of sales and \\nuse taxes. A sales tax is a tax an individual pays upon purchase of a tangible item.  A use \\ntax is a tax an individual pays to utilize, store, or consume an item in a particular location. \\nThe individual who purchases the item is ge nerally responsible for complying with all \\napplicable use-tax laws. A sale s tax differs from a use tax in  that retailers (as opposed to \\nthe buyer) are generally held responsible for collecting and enforci ng the sales tax. When \\nan individual buys and does not pay 